In [64]:
import pandas as pd 
import glob
import re

In [154]:
# Take year from which the stats are displaying
def extract_year(string):
    match = re.match(".+(\d{4})", string)
    if match != None: 
        return match.group(1)

# Set path for where to look for the raw data files
path_pass = "./data/passing/"
path_rush = "./data/rushing/"
all_pass_files = glob.glob(path_pass + "20**-pass.csv")
all_rush_files = glob.glob(path_rush + "20**-rush.csv")

In [155]:
# Clean passing datasets
def clean_pass(p):
    df = pd.DataFrame(p)
    # Remove completely empty rows
    df.dropna(axis=0, how='all', inplace=True)
    # Remove special characters in front of player names
    df['player'] = df['player'].str.rstrip('*+')
    # Add MVP status column
    df.insert(2, "mvp", 0)
    # Renaming columns
    df.rename(columns={"qbrec": "record", "yds": "yards", "tm": "team"}, inplace=True)
    # Split the record column into Wins, Losses and Draws
    df[['wins', 'losses', 'draws']] = df['record'].str.split('-', 3, expand=True)
    df_new = df[['player', 'mvp', 'team', 'wins', 'yards', 'td', 'cmp%', 'int', 'qbr']].copy()
    return df_new

In [156]:
# Clean rushing datasets
def clean_rush(r):
    df = pd.DataFrame(r)
    # Remove completely empty rows
    df.dropna(axis=0, how='all', inplace=True)
    # Remove special characters in front of player names
    df['player'] = df['player'].str.rstrip('*+')
    # Renaming columns
    df.rename(columns={"tm": "team", "yds": "rush yards", "td": "rush td"}, inplace=True)
    df_new = df[['player', 'team', 'rush yards', 'rush td']].copy()
    return df_new

In [157]:
# Create a new CSV files with cleaned datasets

for p in all_pass_files:
    df = pd.read_csv(p, index_col=None, header=0)
    df.columns = map(str.lower, df.columns)
    df = clean_pass(df)
    year = extract_year(p)
    df["year"] = extract_year(p)
    df.to_csv(f'./data/cleaned/{year}-pass.csv', encoding='utf-8', index=False)
    
for r in all_rush_files:
    df = pd.read_csv(r, skiprows=1, index_col=None, header=0)
    df.columns = map(str.lower, df.columns)
    year = extract_year(r)
    df = clean_rush(df)
    df.to_csv(f'./data/cleaned/{year}-rush.csv', encoding='utf-8', index=False)

In [159]:
# Set path for where to look for the cleaned data files
path = "./data/cleaned/"
all_files = glob.glob(path + "20**-pass.csv")

# Loop through the folder and find matching CSV files by year
for file in all_files:
    year = extract_year(file)
    df_pass = pd.read_csv(file, index_col=None, header=0) 
    df_add = pd.read_csv(f'./data/cleaned/{year}-rush.csv', index_col=None, header=0)
    # Merge Rushing and Passing stats
    df = pd.merge(df_pass, df_add, on=['player', 'team'])
    # Save aggregate file
    df.to_csv(f'./data/{year}.csv', encoding='utf-8', index=False)

In [162]:
path = "./data/"
all_files = glob.glob(path + "20**.csv")

mvp = {'2010': 'Tom Brady', '2011': 'Aaron Rodgers', '2013': 'Peyton Manning', '2014': 'Aaron Rodgers', '2015': 'Cam Newton', '2016': 'Matt Ryan', '2017': 'Tom Brady', '2018': 'Patrick Mahomes', '2019': 'Lamar Jackson', '2020': 'Aaron Rodgers'}

# Update each CSV file to indicate whether they won the MVP: 1 = 'yes' and 0 = 'no'
for file in all_files:
    df = pd.read_csv(file, index_col=None, header=0)
    year = extract_year(file)
    winner = mvp[year]
    df.loc[df['player'] == f'{winner}', 'mvp'] = 1
    df_new = df[['year', 'player', 'mvp', 'team', 'wins', 'yards', 'td', 'cmp%', 'int', 'qbr', 'rush yards', 'rush td']].copy()
    df_new.to_csv(file, encoding='utf-8', index=False)